In [21]:
import numpy as np
from utils.formats import load_hdf, save_hdf

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import PreTrainedModel, AutoModel, AutoConfig, AutoModelForSequenceClassification # Or your task-specific model
from transformers import AdamW
from transformers.configuration_utils import PretrainedConfig

ImportError: cannot import name 'AdamW' from 'transformers' (/home/bernard/miniconda3/envs/cs4248_project/lib/python3.11/site-packages/transformers/__init__.py)

### Custom Dataset with Pre-Computed Embeddings

In [ ]:
class EmbeddingDataset(Dataset):
    def __init__(self, embeddings, labels):
        """
        Args:
            embeddings: Pre-computed embeddings (numpy array or torch tensor)
                       Shape: (num_samples, seq_length, embedding_dim)
            labels: Corresponding labels
        """
        self.embeddings = torch.tensor(embeddings, dtype=torch.float)
        self.labels = torch.tensor(labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'input_embeddings': self.embeddings[idx],
            'labels': self.labels[idx]
        }

In [2]:
input_embedding = load_hdf("data/conceptnet_api/retrofit/test_retrofitted")
input_embedding.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
/c/en/chair_meeting,-4.221157e-17,6.753685e-16,1.230684e-17,2.004569e-07,-3.139845e-07,6.492718e-08,-5.941687e-16,-7.451524e-16,-3.245492e-16,1.050718e-16,...,-0.029592,0.028524,-0.038507,-0.051363,-0.052692,-0.054144,-0.052064,-0.045782,-0.061692,0.096738
/c/en/chairperson,-4.212794e-17,6.804785e-16,1.320734e-17,1.889603e-07,-2.959769e-07,6.120348e-08,-6.157436e-16,-6.563820e-16,-3.615849e-16,1.146372e-16,...,-0.027895,0.026888,-0.036298,-0.048417,-0.049670,-0.051038,-0.049078,-0.043156,-0.058154,0.091190
/c/en/chair,-8.906453e-17,6.522849e-16,-2.717981e-17,1.983456e-07,-3.106775e-07,6.424335e-08,-8.001632e-16,-1.145839e-16,-6.617410e-16,2.461867e-16,...,-0.029281,0.028224,-0.038101,-0.050822,-0.052137,-0.053573,-0.051516,-0.045300,-0.061043,0.095720
/c/en/chairperson/n,-9.113447e-17,6.594680e-16,-2.027064e-17,1.867658e-07,-2.925395e-07,6.049270e-08,-7.669644e-16,-2.256429e-16,-6.009574e-16,2.290223e-16,...,-0.027571,0.026576,-0.035877,-0.047855,-0.049093,-0.050446,-0.048508,-0.042655,-0.057479,0.090131
/c/en/president/n/wn/person,2.159347e-15,-3.783949e-16,-5.054159e-15,1.601425e-07,-2.508382e-07,5.186957e-08,-3.338260e-15,-2.859790e-16,3.288175e-15,4.581335e-15,...,-0.023641,0.022788,-0.030763,-0.041033,-0.042095,-0.043255,-0.041593,-0.036575,-0.049285,0.077283


In [7]:
embedding_array = input_embedding.to_numpy()
embedding_array

array([[-4.2211573e-17,  6.7536854e-16,  1.2306839e-17, ...,
        -4.5781989e-02, -6.1692268e-02,  9.6738428e-02],
       [-4.2127938e-17,  6.8047853e-16,  1.3207345e-17, ...,
        -4.3156303e-02, -5.8154099e-02,  9.1190293e-02],
       [-8.9064528e-17,  6.5228493e-16, -2.7179813e-17, ...,
        -4.5299806e-02, -6.1042514e-02,  9.5719561e-02],
       ...,
       [ 7.7134235e-17,  9.1936533e-17,  3.2419220e-16, ...,
        -4.3782692e-02, -5.8998175e-02,  9.2513867e-02],
       [ 7.0881557e-17,  8.9736795e-17,  3.1857404e-16, ...,
        -4.3782692e-02, -5.8998175e-02,  9.2513867e-02],
       [ 9.9147145e-17,  9.2629253e-17,  3.3545909e-16, ...,
        -4.3782692e-02, -5.8998175e-02,  9.2513867e-02]],
      shape=(4081, 300), dtype=float32)

In [10]:
def convert_labels_to_int(labels):
    """
    Convert various label formats to integer labels
    
    Args:
        labels: Could be strings, one-hot, etc.
    
    Returns:
        numpy array of integer labels
    """
    if isinstance(labels[0], str):
        # String labels to integers
        unique_labels = sorted(set(labels))
        label_to_int = {label: i for i, label in enumerate(unique_labels)}
        return np.array([label_to_int[label] for label in labels])
    elif len(labels.shape) > 1 and labels.shape[1] > 1:
        # One-hot to integers
        return np.argmax(labels, axis=1)
    else:
        # Already integers or binary
        return labels.astype(int)
    
labels = convert_labels_to_int(input_embedding.index.to_list())
labels

array([ 593,  595,  590, ..., 2518, 2514, 2516], shape=(4081,))

In [11]:
class EmbeddingDataset(Dataset):
    def __init__(self, embeddings, labels):
        """
        Args:
            embeddings: Pre-computed embeddings (numpy array or torch tensor)
                       Shape: (num_samples, seq_length, embedding_dim)
            labels: Corresponding labels
        """
        self.embeddings = torch.tensor(embeddings, dtype=torch.float)
        self.labels = torch.tensor(labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'input_embeddings': self.embeddings[idx],
            'labels': self.labels[idx]
        }
    
dataset = EmbeddingDataset(embedding_array, labels)

In [22]:
class EmbeddingConfig(PretrainedConfig):
    def __init__(self, embedding_dim=300, **kwargs):
        super().__init__(**kwargs)
        self.embedding_dim = embedding_dim

class EmbeddingModel(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.transformer = AutoModel.from_pretrained("bert-base-uncased")
        self.classifier = nn.Linear(config.embedding_dim, config.num_labels)
        
    def forward(self, input_embeddings, attention_mask=None, labels=None):
        # Bypass the embedding layer and use pre-computed embeddings
        outputs = self.transformer(
            inputs_embeds=input_embeddings,
            attention_mask=attention_mask
        )
        
        # Use [CLS] token for classification
        logits = self.classifier(outputs.last_hidden_state[:, 0, :])
        
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, self.config.num_labels), labels.view(-1))
            
        return {
            'loss': loss,
            'logits': logits,
            'hidden_states': outputs.hidden_states
        }

# Initialize model
config = EmbeddingConfig(embedding_dim=300, num_labels=4081)
model = EmbeddingModel(config)

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

model.train()
for epoch in range(3):
    for batch in train_loader:
        optimizer.zero_grad()
        
        inputs = batch['input_embeddings'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_embeddings=inputs, labels=labels)
        loss = outputs['loss']
        loss.backward()
        optimizer.step()
        
        print(f"Loss: {loss.item()}")

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
class EmbeddingTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def _prepare_inputs(self, inputs):
        prepared = {}
        for k, v in inputs.items():
            if k == 'input_embeddings':
                prepared['inputs_embeds'] = v.to(self.args.device)
            elif isinstance(v, torch.Tensor):
                prepared[k] = v.to(self.args.device)
            else:
                prepared[k] = v
        return prepared

# Usage with TrainingArguments
training_args = TrainingArguments(
    output_dir="./embedding_results",
    per_device_train_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
)

trainer = EmbeddingTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)
trainer.train()